In [1]:
import torchvision.datasets as dsets
from torchvision import transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.manual_seed(0)
num_epoch=20
batch_size=32
device='cuda'
data_root='./data'

import allocateGPU
allocateGPU.allocate_gpu()

Number of gpu avaliable:	1
Current GPU:	0
GPU name: 	GeForce RTX 2080 Ti


In [2]:
attacks=['no_attacks','omniscient','label_flipping','omniscient_aggresive']
attacker_list_labelflipping={'no_attacks':[],'omniscient':[],'label_flipping':[0],'omniscient_aggresive':[]}
attacker_list_omniscient={'no_attacks':[],'omniscient':[0],'label_flipping':[],'omniscient_aggresive':[0]}

In [3]:
class Agg_dataset(torch.utils.data.Dataset):
    '''
        denote n be the number of clients,
        each entry of dataset is a 2-tuple of (weight delta, labels):= (1 x n tensor, 1 x n tensor)
        honest clients are labeled 1, malicious clients are labeled 0
    '''
    def __init__(self,path,attacks):
        super(Agg_dataset).__init__() 
        data=torch.load(path,map_location='cpu')
        data_tensors=torch.cat([data[param] for param in data],0)
        self.data=data_tensors
        self.label=attacks
        label=attacks/torch.sum(attacks)
        self.center=torch.sum(data_tensors*label,1).view(-1,1)
        self.num_clients=attacks.shape[0]
        self.n=10000
        dimension=100
        self.indexes=torch.randint(self.data.shape[0],(self.n,100))
        
    def __getitem__(self, index):
#         

        data_out=self.data[self.indexes[index]]
        label_out=self.label
        center_out=self.center[self.indexes[index]]
        perm=torch.randperm(self.num_clients)
#         data_sorted=torch.sort(data_out)
#         perm=data_sorted[1]
        data_out_shuffled=torch.index_select(data_out, -1, perm)

        return data_out_shuffled, [label_out[perm],center_out]
#         return data_out,[label_out,center_out]
    def __len__(self):
#         return self.data.shape[0]//100
        return self.n

In [4]:
def get_loader(attack):
    label=torch.ones(10)
    for i in attacker_list_labelflipping[attack]:
        label[i]=0
    for i in attacker_list_omniscient[attack]:
        label[i]=0
    path=f'./AggData/{attack}/FedAvg_0.pt'
    dataset=Agg_dataset(path,label)
    validation_split = .2
    shuffle_dataset = True
    random_seed= 42

    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
    valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)
    return train_loader, test_loader

In [5]:
def train(net,train_loader,criterion,optimizer,device, on):
    net.to(device)
    for idx, (data,target) in enumerate(train_loader):
        data = data.to(device)
        target = target[on].to(device)
        optimizer.zero_grad()   
        output = net(data)
        loss = criterion(output[on], target)

        loss.backward()
        optimizer.step()
    
def test(net,test_loader,device,message_prefix):
    net.to(device)
    accuracy = 0
    accuracy_mean = 0
    accuracy_median = 0
    count = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target[1].to(device)
            outputs = net(data)
            accuracy+=F.l1_loss(outputs[1], target)
            accuracy_mean+=F.l1_loss(data.mean(-1).unsqueeze(-1), target)
            accuracy_median+=F.l1_loss(data.median(-1)[0].unsqueeze(-1), target)
            count+=len(data)
    print('Accuracy: %.4E (%.4E, %.4E)' % (accuracy/count,accuracy_mean/count,accuracy_median/count ))
    return accuracy/count, accuracy_mean/count, accuracy_median/count

In [6]:
# torch.mean(batch[0],-1).unsqueeze(-1).shape

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
def dot_product(A,B):
    return torch.bmm(A.view(A.shape[0],1,A.shape[1]),B.view(B.shape[0],B.shape[1],1))
class Mlp(nn.Module):
    def __init__(self,in_dim, n ,m ):
        super(Mlp, self).__init__()
        self.in_dim=in_dim
        self.n = n
        self.fc1 = nn.Linear(self.in_dim*n, m)
        self.fc2 = nn.Linear(m, m)
        self.fc3 = nn.Linear(m, self.n)

    def forward(self, input):
        x = input.view(-1, self.in_dim*self.n)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        pred = torch.sum(x.view(-1,1,self.n)*input,dim=-1).unsqueeze(-1)/9
    
        return x,pred

In [8]:
class CNN(nn.Module):
    def __init__(self,n,in_dim):
        super(CNN, self).__init__()
        self.in_dim=in_dim
        self.fc1 = nn.Conv1d(1, n, kernel_size=3,dilation=1, padding=1)
        self.fc2 = nn.Conv1d(n, n, kernel_size=3,dilation=2, padding=2)
        self.fc3 = nn.Conv1d(n, self.in_dim, kernel_size=3,dilation=1, padding=1)
        self.maxpool1=nn.AdaptiveMaxPool1d(1)
        

    def forward(self, input):
        x = input.view(-1, 1, self.in_dim)
        x = nn.LeakyReLU()(self.fc1(x))
        x = nn.LeakyReLU()(self.fc2(x))
        x = nn.LeakyReLU()(self.fc3(x))
        x = self.maxpool1(x)

        x=x.squeeze()
        x = F.softmax(x,dim=1)
        pred=dot_product(x,input).squeeze(-1)
        return x, pred

In [35]:

class block(nn.Module):
    def __init__(self,in_,out_):
        super(block, self).__init__()
        self.main=torch.nn.Sequential(
                    nn.Conv2d(in_, out_, kernel_size=1),
                    torch.nn.BatchNorm2d(out_),
                    nn.ReLU(),
                    )
    def forward(self,x):
        out=self.main(x)
        return out
    
class block_no_activation(nn.Module):
    def __init__(self,in_,out_):
        super(block_no_activation, self).__init__()
        self.main=torch.nn.Sequential(
                    nn.Conv2d(in_, out_, kernel_size=1),
                    torch.nn.BatchNorm2d(out_),
                    )
    def forward(self,x):
        out=self.main(x)
        return out
    
class PointNet(nn.Module):
    def __init__(self,in_dim, n):
        '''
        in_dim:=dimension of weight vector
        n:= number of clients
        '''
        super(PointNet, self).__init__()
        self.in_dim = in_dim
        self.n = n
        self.local = torch.nn.Sequential(
                        block(self.in_dim,64),
                        block(64,64),
                        block(64,64)
                    )
        self.globa = torch.nn.Sequential(
                        block(64,128),
                        block(128,1024),
                        nn.AdaptiveMaxPool2d(1)
                    )
        self.direct_out= block(1024,10)
        self.MLP = torch.nn.Sequential(
                        block(1088,512),
                        block(512,256),
                        block(256,128),
                        nn.Dropout(p=0.7, inplace=True),
                        block_no_activation(128,1)
                      )


        

    def forward(self, input):
#         x = input.view(-1, self.n, self.in_dim)
        x=input.view(-1,self.in_dim,self.n,1)
        for module in self.local:
            x = module(x)
#             print(f'local:\t {x.shape}')
        x_local=x
        for module in self.globa:
            x = module(x)
#             print(f'global:\t {x.shape}')
        x_global=x.repeat(1,1,self.n,1)
#         print(f'tile:\t {x_global.shape}')
        x=torch.cat([x_local,x_global],dim=1)
#         print(x.shape)
        for module in self.MLP:
            x = module(x)
#             print(f'MLP:\t {x.shape}')
#         x=self.direct_out(x)
        x=x.squeeze()
#        x = F.softmax(x,dim=1)
        x = torch.sigmoid(x)
#         pred=dot_product(input,x).squeeze(-1)
#        x2= F.softmax(x,dim=1)
        x3 = (x>0.5).float().cuda()
        x3 = x3/torch.sum(x3,-1).view(-1,1)
        pred = torch.sum(x3.view(-1,1,self.n)*input,dim=-1).unsqueeze(-1)

        return x,pred

    def forward2(self, input, n):
#         x = input.view(-1, self.n, self.in_dim)
        x=input.view(-1,self.in_dim,n,1)
        for module in self.local:
            x = module(x)
#             print(f'local:\t {x.shape}')
        x_local=x
        for module in self.globa:
            x = module(x)
#             print(f'global:\t {x.shape}')
        x_global=x.repeat(1,1,n,1)
#         print(f'tile:\t {x_global.shape}')
        x=torch.cat([x_local,x_global],dim=1)
#         print(x.shape)
        for module in self.MLP:
            x = module(x)
#             print(f'MLP:\t {x.shape}')
#         x=self.direct_out(x)
        x=x.squeeze()
#        x = F.softmax(x,dim=1)
        x = torch.sigmoid(x)
#         pred=dot_product(input,x).squeeze(-1)
#        x2= F.softmax(x,dim=1)
        x3 = (x>0.5).float().cuda()
        x3 = x3/torch.sum(x3,-1).view(-1,1)
        pred = torch.sum(x3.view(-1,1,n)*input,dim=-1).unsqueeze(-1)

        return x,pred



In [23]:
from tensorboardX import SummaryWriter
def write(name,scalar):
    writer=SummaryWriter(f'./agg_logs/{name}')
    writer.add_scalar('l1 loss', scalar, 0)
    writer.close()

In [24]:
accuracy_list={}


In [25]:
loaders={attack:get_loader(attack) for attack in attacks}


In [26]:
# torch.mean(batch[0][1],1)
# 0

In [27]:
batch=next(iter(loaders[attacks[1]][0]))
print(batch[0].shape)
print(batch[1][0].shape)
print(batch[1][1].shape)

torch.Size([32, 100, 10])
torch.Size([32, 10])
torch.Size([32, 100, 1])


In [28]:
batch=next(iter(loaders[attacks[1]][0]))
net_p=PointNet(100,10).cuda()
net_p(batch[0].cuda())[0].shape


torch.Size([32, 10])

In [36]:
mode_name=['classification']
for attack in attacks[1:]:
    train_loader, test_loader=loaders[attack]
    for criterion in [torch.nn.BCELoss()]:
        mode=0
        for lr in [0.1,0.01,0.001]:

            net_ptnet=PointNet(100,10)

            for net in [net_ptnet]:
                
                training_alias=f'{attack}/{net.__class__.__name__}/{criterion.__class__.__name__}/lr_{lr}'
                if training_alias in accuracy_list:
                    continue
                
                print('Start training of %s'%training_alias)
                optimizer = optim.Adam(net.parameters(), lr=lr)
                for epoch in range(num_epoch):
                    train(net,train_loader,criterion,optimizer,device,mode)
                    score=test(net,test_loader,device,'')
                write(training_alias,score[0])
                accuracy_list[training_alias]=score[0].item()
    accuracy_list[f'{attack}/mean']=score[1].item()
    write(f'{attack}/mean',score[1])
    accuracy_list[f'{attack}/median']=score[2].item()
    write(f'{attack}/median',score[1])

Start training of omniscient/PointNet/BCELoss/lr_0.1
Accuracy: 5.2091E-08 (1.8397E-05, 5.9228E-07)


KeyboardInterrupt: 

In [ ]:
# net=Mlp(100,10,10).cuda()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
for i in range(1000):
    criterion=torch.nn.BCELoss()
    data=batch[0].cuda()
    target=batch[1][0].cuda()
    optimizer.zero_grad()   
    output = net(data)
    loss = criterion(output[0], target)

    loss.backward()
    optimizer.step()
print(output[0][0])
print(target[0])
print(loss)

In [31]:
batch[1][0]

tensor([[1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],


In [37]:
net(batch[0].cuda())

(tensor([[9.9816e-01, 9.9900e-01, 9.9871e-01, 9.9842e-01, 9.9934e-01, 9.9866e-01,
          2.1455e-07, 9.9862e-01, 9.9580e-01, 9.9952e-01],
         [9.9891e-01, 9.9839e-01, 9.9895e-01, 9.9945e-01, 9.9878e-01, 9.9860e-01,
          9.9788e-01, 9.9873e-01, 1.1962e-08, 9.9872e-01],
         [9.9912e-01, 3.8677e-04, 9.9782e-01, 9.9905e-01, 9.9783e-01, 9.9726e-01,
          9.9831e-01, 9.9848e-01, 9.9853e-01, 9.9943e-01],
         [9.9843e-01, 9.9843e-01, 9.9896e-01, 9.9796e-01, 1.1561e-06, 9.9821e-01,
          9.9698e-01, 9.9864e-01, 9.9840e-01, 9.9917e-01],
         [9.9920e-01, 9.9841e-01, 9.9891e-01, 9.9847e-01, 9.9805e-01, 9.9908e-01,
          9.9895e-01, 9.9904e-01, 5.4425e-06, 9.9925e-01],
         [9.9814e-01, 9.8365e-04, 9.9690e-01, 9.9894e-01, 9.9899e-01, 9.9739e-01,
          9.9848e-01, 9.9849e-01, 9.9921e-01, 9.9893e-01],
         [9.9858e-01, 9.9924e-01, 9.9808e-01, 9.9913e-01, 9.9927e-01, 9.9691e-01,
          9.9876e-01, 9.9781e-01, 9.9895e-01, 5.1436e-08],
         [9.9

In [38]:
small_batch=torch.index_select(batch[0],-1,torch.LongTensor([0,1,2,3,4]))
net.forward2(small_batch.cuda(),5)

(tensor([[9.9808e-01, 9.9851e-01, 9.9890e-01, 9.9914e-01, 9.9808e-01],
         [9.9888e-01, 9.9892e-01, 9.9781e-01, 9.9868e-01, 9.9878e-01],
         [9.9784e-01, 2.3904e-07, 9.9895e-01, 9.9811e-01, 9.9809e-01],
         [9.9809e-01, 9.9923e-01, 9.9914e-01, 9.9753e-01, 1.0071e-04],
         [9.9784e-01, 9.9717e-01, 9.9828e-01, 9.9797e-01, 9.9873e-01],
         [9.9903e-01, 2.2263e-09, 9.9897e-01, 9.9911e-01, 9.9909e-01],
         [9.9864e-01, 9.9835e-01, 9.9916e-01, 9.9772e-01, 9.9819e-01],
         [9.9918e-01, 9.9797e-01, 9.9896e-01, 1.8374e-02, 9.9911e-01],
         [9.9777e-01, 9.9955e-01, 9.9229e-01, 9.9905e-01, 9.9629e-01],
         [2.4453e-04, 9.9890e-01, 9.9839e-01, 9.9883e-01, 9.9816e-01],
         [9.9905e-01, 9.9904e-01, 9.9743e-01, 2.4812e-07, 9.9784e-01],
         [9.9886e-01, 9.9891e-01, 9.9808e-01, 9.9775e-01, 9.9906e-01],
         [9.9889e-01, 3.3394e-06, 9.9891e-01, 9.9943e-01, 9.9896e-01],
         [1.7978e-05, 9.9895e-01, 9.9785e-01, 9.9846e-01, 9.9857e-01],
      

## Compare the results

In [ ]:
accuracy_list

In [ ]:
import pandas as pd
from collections import OrderedDict  

In [ ]:
struct=[key.split('/',1)+[accuracy_list[key]] for key  in accuracy_list.keys()]
df=pd.DataFrame(np.array([row[2] for row in struct]).reshape(4,6))


index=list(OrderedDict.fromkeys([row[0] for row in struct]))
df.index=index

columns=list(OrderedDict.fromkeys([row[1] for row in struct]))
df.columns=columns

In [ ]:
df

In [ ]:
def highlight_max(s):
    '''
    highlight the minimum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

print("\nHighlight the minimum value in each column:")
df.T.style.apply(highlight_max,subset=pd.IndexSlice[:, df.T.columns])

## Visualize distribution of gradients

In [ ]:
attack=attacks[1]
label=torch.ones(10)
for i in attacker_list_labelflipping[attack]:
    label[i]=0
for i in attacker_list_omniscient[attack]:
    label[i]=0
path=f'./AggData/{attack}/FedAvg_0.pt'
dataset=Agg_dataset(path,label)
dist=dataset.center.numpy()

In [ ]:
df=pd.DataFrame(np.log(np.abs(dist))).replace([np.inf, -np.inf], np.nan)

In [ ]:
df.hist()

In [ ]:
df.quantile([0.25,0.75])